In [13]:
import pandas as pd
import numpy as np
import re
import pickle as pkl

In [2]:
valid_name = "../output/valid_compare_16.v3.csv"

In [3]:
valid_df = pd.read_csv(valid_name)

In [4]:
valid_df

,before,true,class_pred
0,washingtonpost.com,ELECTRONIC,PLAIN
1,OWN,PLAIN,LETTERS
2,07,DIGIT,CARDINAL
3,SoundersFC.com,ELECTRONIC,PLAIN
4,OAM,LETTERS,PLAIN
5,utenlandskorrespondentene,PLAIN,ELECTRONIC
6,-,PLAIN,PUNCT
7,HybridCars.com,ELECTRONIC,PLAIN
8,1990,CARDINAL,DATE
9,1992,CARDINAL,DATE


In [5]:
labels = ['PLAIN', 'PUNCT', 'DATE', 'LETTERS', 'CARDINAL', 'VERBATIM',
          'DECIMAL', 'MEASURE', 'MONEY', 'ORDINAL', 'TIME', 'ELECTRONIC',
          'DIGIT', 'FRACTION', 'TELEPHONE', 'ADDRESS']
label2index = dict(zip(labels, range(len(labels))))
index2label = dict([(v, k) for k, v in label2index.items()])

In [6]:
def is_com(token):
    token = str(token)
    pt = re.compile(r"\.[a-zA-Z]{2,}")
    return True if pt.search(token) else False

In [7]:
count = 0
for i, row in valid_df.iterrows():
    # 1.被误认为是cardinal的实际上的date的年份
    if row["class_pred"] == "CARDINAL":
        if row["before"].isdigit() and int(row["before"]) <= 3000 and int(row["before"]) >= 1000:
            # 不能直接修改row，因为row是副本
            valid_df.loc[i, "class_pred"] = "DATE"
            count += 1
    # 2.被误认为是plain的实际上的electronic的网站
    elif row["class_pred"] == "PLAIN":
        if is_com(row["before"]):
            valid_df.loc[i, "class_pred"] = "ELECTRONIC"
            count += 1
print(count)

66


In [8]:
diff_df = valid_df.loc[valid_df["true"] != valid_df["class_pred"]]

In [9]:
diff_df.groupby("true").agg(['count'])

,before,class_pred
,count,count
true,,
ADDRESS,2,2
CARDINAL,130,130
DATE,12,12
DECIMAL,16,16
DIGIT,72,72
ELECTRONIC,4,4
FRACTION,2,2
LETTERS,653,653


In [17]:
diff_df.loc[diff_df["true"] == "CARDINAL"]

,before,true,class_pred
8,1990,CARDINAL,DATE
9,1992,CARDINAL,DATE
14,I,CARDINAL,PLAIN
17,1904,CARDINAL,DATE
22,1978,CARDINAL,DATE
35,2012,CARDINAL,DATE
37,2011,CARDINAL,DATE
51,1977,CARDINAL,DATE
52,1942,CARDINAL,DATE
61,CLXXVI,CARDINAL,LETTERS


In [19]:
diff_df.loc[np.logical_and(diff_df["true"] == "DIGIT", diff_df["class_pred"] == "CARDINAL")]

,before,true,class_pred
2,07,DIGIT,CARDINAL
21,9,DIGIT,CARDINAL
25,2,DIGIT,CARDINAL
60,8123,DIGIT,CARDINAL
76,41,DIGIT,CARDINAL
83,5,DIGIT,CARDINAL
95,3,DIGIT,CARDINAL
158,402,DIGIT,CARDINAL
186,201,DIGIT,CARDINAL
217,479,DIGIT,CARDINAL


In [24]:
diff_df.loc[diff_df["true"] == "LETTERS"]

,before,true,class_pred
4,OAM,LETTERS,PLAIN
10,fl,LETTERS,PLAIN
13,SES,LETTERS,PLAIN
15,r,LETTERS,PLAIN
18,zkp,LETTERS,PLAIN
19,Dy,LETTERS,PLAIN
20,Cik,LETTERS,PLAIN
31,UHD,LETTERS,PLAIN
32,PISSLRE,LETTERS,PLAIN
33,ydow,LETTERS,PLAIN


In [10]:
lp_df = diff_df.loc[np.logical_and(diff_df["true"] == "LETTERS", diff_df["class_pred"] == "PLAIN")]
lp_df.shape

(643, 3)

In [11]:
pl_df = diff_df.loc[np.logical_and(diff_df["true"] == "PLAIN", diff_df["class_pred"] == "LETTERS")]
pl_df.shape

(377, 3)

In [14]:
with open("../output/letter_set.pkl", "rb") as f:
    letter_set = pkl.load(f)
len(letter_set)

224397

In [15]:
count = 0
for one in lp_df["before"].values:
    if one in letter_set:
        count += 1
print(count)

590


In [16]:
count = 0
for one in pl_df["before"].values:
    if one in letter_set:
        count += 1
print(count)

0
